<a href="https://colab.research.google.com/github/umedadada/nikkei_software_09/blob/add_code2/nikkei_software_09_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import pandas as pd
import glob
import plotly.express as px
import datetime

In [96]:
# ファイル名の取得
files = glob.glob("./expenses/*.csv")
# 空のデータフレームを用意する
df = pd.DataFrame()
# ファイルの読み込みと結合
for file in files:
    tmp=pd.read_csv(file,encoding='S-JIS')
    df=pd.concat([df,tmp])

In [97]:
df.head()

,計算対象,日付,内容,金額（円）,保有金融機関,大項目,中項目,メモ,振替,ID
0,1,2022/03/28,マネフォストア,-13140,マネフォ銀行,健康・医療,フィットネス,NaN,0,437f-bb85-2d3ab873125f
1,0,2022/03/28,マネフォストア,-15588,マネフォ銀行,現金・カード,カード引き落とし,NaN,1,4948-8fea-a299937b23a9
2,1,2022/03/25,マネフォストア,-832,マネフォ銀行,食費,食料品,NaN,0,4ec3-bf74-536b810ce905
3,1,2022/03/24,マネフォストア,-2923,マネフォ銀行,日用品,ドラッグストア,NaN,0,4620-9f13-d53aa8e9da08
4,1,2022/03/24,マネフォストア,-2333,マネフォ銀行,食費,外食,NaN,0,4f2f-aecc-5bb23d29fe2b


In [98]:
# 分析に不要な列を削除する
df=df.drop(['計算対象','内容','保有金融機関','メモ','振替','ID'],axis=1)
# カラム名を英語に変更する
df.columns=['date','amount','item','sub_item']
# 支出データだけを抽出して、正負を変える
## 支出データにだけ絞る
df=df[df['amount']<0]
## 正負を逆転させる
df['amount']=-1*df['amount']
## 不要なデータを除外する
df=df[df['item'] != '現金・カード']
df=df[df['item'] != '住宅']
df=df[df['item'] != '未分類']
# 日時カラムの追加
## dateカラムの型を変更する
df['date']=pd.to_datetime(df['date'])
df['month']=df['date'].dt.month
df['week']=df['date'].dt.isocalendar().week
df['dow']=df['date'].dt.dayofweek
# カラムの整頓
df=df.reindex(columns=['date', 'month','week', 'dow','item','sub_item','amount'])
# 取引日順に並び替える
df=df.sort_values('date')

In [99]:
print(df.info)#データの内容とサイズの確認
print(df.dtypes)#変数の型の確認

<bound method DataFrame.info of          date  month  week  dow   item sub_item  amount
84 2021-04-01      4    13    3     食費      食料品     201
81 2021-04-01      4    13    3     食費      食料品     100
83 2021-04-01      4    13    3    交通費      交通費     980
82 2021-04-01      4    13    3     食費      食料品     140
80 2021-04-02      4    13    4     食費       外食    1100
..        ...    ...   ...  ...    ...      ...     ...
5  2022-03-24      3    12    3     食費      食料品     272
4  2022-03-24      3    12    3     食費       外食    2333
3  2022-03-24      3    12    3    日用品  ドラッグストア    2923
2  2022-03-25      3    12    4     食費      食料品     832
0  2022-03-28      3    13    0  健康・医療   フィットネス   13140

[853 rows x 7 columns]>
date        datetime64[ns]
month                int64
week                UInt32
dow                  int64
item                object
sub_item            object
amount               int64
dtype: object


In [100]:
fig = px.histogram(df, x="amount", marginal="box")
fig.show()

In [101]:
print(df['amount'].describe()) #要約統計量の表示
print('median   ' + str(df['amount'].median())) #describe関数に中央値の出力がないため、別途表示

count      853.000000
mean      2343.798359
std       3918.080465
min          4.000000
25%        389.000000
50%        829.000000
75%       2486.000000
max      42980.000000
Name: amount, dtype: float64
median   829.0


In [102]:
fig = px.box(df, x='month', y="amount",color='month')
fig.show()

In [103]:
df.head()

,date,month,week,dow,item,sub_item,amount
84,2021-04-01,4,13,3,食費,食料品,201
81,2021-04-01,4,13,3,食費,食料品,100
83,2021-04-01,4,13,3,交通費,交通費,980
82,2021-04-01,4,13,3,食費,食料品,140
80,2021-04-02,4,13,4,食費,外食,1100


In [104]:
# 縦持ちを横持ちに変換する
pivot_df = df.pivot_table(values=['amount'], index=['week'], columns=['item'], aggfunc='sum')
# 欠損部分にゼロを代入する
pivot_df=pivot_df.fillna(0)
# multi indexを解除する
pivot_df.columns=pivot_df.columns.droplevel(0)

In [105]:
pivot_df.head()

item,その他,交通費,交際費,健康・医療,教養・教育,日用品,水道・光熱費,自動車,衣服・美容,趣味・娯楽,通信費,食費
week,,,,,,,,,,,,
1,0.0,374.0,0.0,0.0,0.0,1621.0,0.0,0.0,0.0,555.0,1180.0,41650.0
2,0.0,4500.0,0.0,0.0,0.0,1074.0,2113.0,0.0,0.0,0.0,0.0,6827.0
3,210.0,0.0,1273.0,0.0,0.0,0.0,0.0,0.0,2486.0,2550.0,0.0,16002.0
4,0.0,0.0,0.0,13140.0,0.0,6671.0,0.0,0.0,0.0,3399.0,0.0,6261.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3148.0,880.0,0.0,4700.0,385.0,6345.0


In [106]:
fig = px.scatter_matrix(pivot_df,dimensions=pivot_df.columns)
fig.show()

In [107]:
corr=pivot_df.corr()
corr

item,その他,交通費,交際費,健康・医療,教養・教育,日用品,水道・光熱費,自動車,衣服・美容,趣味・娯楽,通信費,食費
item,,,,,,,,,,,,
その他,1.000000,0.156899,-0.040164,-0.037626,-0.079857,-0.013203,-0.037730,-0.003724,-0.045051,0.089083,0.005105,-0.127898
交通費,0.156899,1.000000,0.057692,0.277064,0.032548,-0.098432,-0.061598,0.047884,0.171378,0.501254,0.210346,-0.045204
交際費,-0.040164,0.057692,1.000000,-0.049373,-0.012839,0.011955,-0.049320,0.031937,0.586555,-0.054943,0.006184,0.214648
健康・医療,-0.037626,0.277064,-0.049373,1.000000,-0.147325,-0.074472,-0.089857,-0.021536,-0.116720,-0.028951,-0.093893,0.069207
教養・教育,-0.079857,0.032548,-0.012839,-0.147325,1.000000,-0.053976,-0.040399,-0.094636,0.147161,0.023648,-0.114212,0.135531
日用品,-0.013203,-0.098432,0.011955,-0.074472,-0.053976,1.000000,-0.053613,0.107013,0.139769,-0.121355,0.315795,0.192397
水道・光熱費,-0.037730,-0.061598,-0.049320,-0.089857,-0.040399,-0.053613,1.000000,-0.052443,-0.057072,-0.046659,0.522931,-0.084662
自動車,-0.003724,0.047884,0.031937,-0.021536,-0.094636,0.107013,-0.052443,1.000000,-0.113233,0.018274,0.327928,0.056547
衣服・美容,-0.045051,0.171378,0.586555,-0.116720,0.147161,0.139769,-0.057072,-0.113233,1.000000,-0.075732,-0.015540,-0.039472


In [108]:
fig = px.imshow(corr, text_auto=True)
fig.show()